In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from lightgbm import LGBMClassifier
from numpy import mean
from numpy import std
import sklearn

In [15]:
temp = pd.DataFrame({"A": [1,2,3], "B": [2,3,4], "C": [1,4,5]})
temp.mode(axis=1)

,0,1,2
0,1.0,NaN,NaN
1,2.0,3.0,4.0
2,3.0,4.0,5.0


In [3]:
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
df1.drop(columns = ['Veteran flag','period mean','period std',
                    'Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)


path = '/home/daisy/FDA_Dataset/inpatient_all_final_2.csv'
df2 = pd.read_csv(path).iloc[:,1:]
df2.drop(columns = ['Veteran flag','period mean','period std',
                    'Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)

In [4]:
X_admission = df1.drop(columns = ['Readmission','Died'])
Y_admission = df1[['Readmission']]

X_mortality = df1.drop(columns = ['Died'])
Y_mortality = df1[['Died']]

In [6]:
X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(X_admission, Y_admission, test_size=0.20, random_state=42)
X_train_mor, X_test_mor, y_train_mor, y_test_mor = train_test_split(X_mortality, Y_mortality, test_size=0.20, random_state=42)


In [7]:
# Filling continuous features with mean values
numeric_cols = df1.columns[df1.isna().any()].tolist()

X_train_ad[numeric_cols] = X_train_ad[numeric_cols].fillna(X_train_ad[numeric_cols].mean())
X_train_mor[numeric_cols] =  X_train_mor[numeric_cols].fillna(X_train_mor[numeric_cols].mean())

X_test_ad[numeric_cols] = X_test_ad[numeric_cols].fillna(X_test_ad[numeric_cols].mean())
X_test_mor[numeric_cols] = X_test_mor[numeric_cols].fillna(X_test_mor[numeric_cols].mean())

### Model

#### XGBoost

In [27]:
import xgboost as xgb

In [28]:
# readmission
xgb_model_ad = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(xgb_model_ad, X_train_ad, y_train_ad, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

xgb_model_ad.fit(X_train_ad, y_train_ad)
y_pred_ad = xgb_model_ad.predict(X_test_ad)

print(classification_report(y_test_ad, y_pred_ad, target_names= ['Not Readmitted', 'Readmitted']))

# mortality
xgb_model_mor = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(xgb_model_mor, X_train_mor, y_train_mor, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

xgb_model_mor.fit(X_train_mor, y_train_mor)
y_pred_mor = xgb_model_mor.predict(X_test_mor)

print(classification_report(y_test_mor, y_pred_mor, target_names= ['Not Dead', 'Dead']))

Accuracy: 1.000 (0.000)
                precision    recall  f1-score   support

Not Readmitted       1.00      1.00      1.00      3316
    Readmitted       1.00      1.00      1.00     13592

      accuracy                           1.00     16908
     macro avg       1.00      1.00      1.00     16908
  weighted avg       1.00      1.00      1.00     16908

Accuracy: 0.842 (0.003)
              precision    recall  f1-score   support

    Not Dead       0.87      0.95      0.91     13744
        Dead       0.63      0.39      0.48      3164

    accuracy                           0.84     16908
   macro avg       0.75      0.67      0.69     16908
weighted avg       0.83      0.84      0.83     16908



#### AdaBoost

In [25]:
from sklearn.ensemble import AdaBoostClassifier
# Readmission
adaboost_ad = AdaBoostClassifier(n_estimators=100, random_state=0)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(adaboost_ad, X_train_ad, y_train_ad, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
adaboost_ad.fit(X_train_ad, y_train_ad)
y_pred_ad = adaboost_ad.predict(X_test_ad)

print(classification_report(y_test_ad, y_pred_ad, target_names= ['Not Readmitted', 'Readmitted']))

# Mortality
adaboost_mor = AdaBoostClassifier(n_estimators=100, random_state=0)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(adaboost_mor, X_train_mor, y_train_mor, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
adaboost_mor.fit(X_train_mor, y_train_mor)
y_pred_mor = adaboost_mor.predict(X_test_mor)

print(classification_report(y_test_mor, y_pred_mor, target_names= ['Not Dead', 'Dead']))

                precision    recall  f1-score   support

Not Readmitted       1.00      1.00      1.00      3316
    Readmitted       1.00      1.00      1.00     13592

      accuracy                           1.00     16908
     macro avg       1.00      1.00      1.00     16908
  weighted avg       1.00      1.00      1.00     16908

Accuracy: 0.833 (0.003)
              precision    recall  f1-score   support

    Not Dead       0.86      0.95      0.90     13744
        Dead       0.61      0.31      0.41      3164

    accuracy                           0.83     16908
   macro avg       0.73      0.63      0.66     16908
weighted avg       0.81      0.83      0.81     16908



#### LGBM

In [24]:
y_train_ad = np.ravel(sklearn.utils.validation.column_or_1d(y_train_ad, warn=True))
y_train_mor = np.ravel(sklearn.utils.validation.column_or_1d(y_train_mor, warn=True))

# readmission
lgbm_ad = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(lgbm_ad, X_train_ad, y_train_ad, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
lgbm_ad.fit(X_train_ad, y_train_ad)
# make a single prediction
y_pred_ad = lgbm_ad.predict(X_test_ad)
print(classification_report(y_test_ad, y_pred_ad, target_names= ['Not Readmitted', 'Readmitted']))

# motality
lgbm_mor = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(lgbm_mor, X_train_mor, y_train_mor, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
lgbm_mor.fit(X_train_mor, y_train_mor)
# make a single prediction
y_pred_mor = lgbm_mor.predict(X_test_mor)
print(classification_report(y_test_mor, y_pred_mor, target_names= ['Not Dead','Dead']))

Accuracy: 1.000 (0.000)
                precision    recall  f1-score   support

Not Readmitted       1.00      1.00      1.00      3316
    Readmitted       1.00      1.00      1.00     13592

      accuracy                           1.00     16908
     macro avg       1.00      1.00      1.00     16908
  weighted avg       1.00      1.00      1.00     16908

Accuracy: 0.844 (0.003)
              precision    recall  f1-score   support

    Not Dead       0.87      0.96      0.91     13744
        Dead       0.66      0.35      0.46      3164

    accuracy                           0.85     16908
   macro avg       0.76      0.66      0.68     16908
weighted avg       0.83      0.85      0.83     16908

